Resources used:
- https://python.langchain.com/v0.1/docs/integrations/vectorstores/milvus/
- https://milvus.io/docs/integrate_with_langchain.md

In [26]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAPI_KEY") # (Optional), if OpenAI Model is used

MODEL = "mistral:latest" # Name of the model used by Ollama

COLLECTION_NAME = 'scouting' # Name of the Collection to be created

DIMENSION = 1536 # Dimension of the embeddings

URI = 'http://localhost:19530' # Connection parameters for the Milvus Server

In [27]:
from langchain_community.llms import Ollama
from langchain_openai import OpenAIEmbeddings

model = Ollama(model=MODEL)
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [51]:
# Configure the prompt template that is used to ask the LLM

from langchain_core.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer from the context, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
print(prompt.format(context="Here is some context", question="Here is a question"))


Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer from the context, just say that you don't know, don't try to make up an answer.
<context>
Here is some context
</context>

<question>
Here is a question
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:


In [52]:
# Use Milvus as Vectorstore

from langchain_community.vectorstores import Milvus

connection_args = {'uri': URI }

vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name=COLLECTION_NAME,
    vector_field="embeddings",
    primary_field="id",
    auto_id=True
)


In [53]:
# Convert the vector store to a retriever
# k:2 --> Limit to 2 documents
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})
# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [54]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

# Invoke the RAG chain with a specific question and retrieve the response
query = "Ich brauche einen Rechtsverteidiger, der konstant über das ganze Spiel durchspielt. Er sollte auch im Spiel nach vorne gut sein"
res = rag_chain.invoke(query)
res

" Based on the context provided, you are looking for a right defender who consistently performs throughout the game and also contributes offensively. The player mentioned in the context demonstrates this skillset. Here's a summary of his abilities:\n\n- A constant performance over 90 minutes (implies consistent play)\n- Secure in defense, allowing few opportunities for flanks or breaks (implies good defensive skills)\n- Offensively marks territory with precise passes and timed advances (implies good ball control and decision-making skills)\n- Notable endurance and high readiness to act (implies he can maintain his performance level throughout the game)\n- Lacks top speed, which could be a weakness against quick wing players (implies potential limitations in handling fast opponents)\n- Still has room for improvement at 25 years old (implies he has growth potential and may improve further with coaching and experience)"

In [32]:
from langchain_core.documents.base import Document

def extract_metadata(doc: Document) -> dict:
    return doc.metadata

In [43]:
# How to work with meta data from our query
retrived_documents = retriever.invoke(query)
retrived_documents

[Document(page_content='\n Wieder als rechter Verteidiger eingesetzt, zeigte er eine beständige Leistung über die gesamte Spielzeit.\nDefensiv sehr solide, ließ kaum Flanken und gegnerische Durchbrüche zu.\nOffensiv setzte er Akzente durch präzise Flanken und gute Vorstöße.\nSeine Ausdauer und Einsatzbereitschaft waren erneut beeindruckend.\nSeine Endgeschwindigkeit bleibt eine Schwäche, die gegen schnelle Gegenspieler problematisch sein könnte.\nMit 25 Jahren hat er noch Verbesserungspotenzial.\n ', metadata={'id': 450236041423488387, 'scout_id': '0987', 'player_id': 'bf1234567890abcdef123456', 'player_transfermarkt_id': '951357', 'grade_rating': 0.699999988079071, 'grade_potential': 0.75}),
 Document(page_content='\n Spielte als rechter Verteidiger und zeigte eine konstante Leistung über 90 Minuten.\nWar sicher in der Defensive und ließ kaum Flanken zu.\nOffensiv setzte er Akzente mit präzisen Pässen und gut getimten Vorstößen.\nSeine Ausdauer und Einsatzbereitschaft waren bemerkensw

In [44]:
metadata = extract_metadata(retrived_documents[0])
metadata

{'id': 450236041423488387,
 'scout_id': '0987',
 'player_id': 'bf1234567890abcdef123456',
 'player_transfermarkt_id': '951357',
 'grade_rating': 0.699999988079071,
 'grade_potential': 0.75}

In [45]:
# Transfermarkt Link
print(f"Transfermarkt.com Link: https://www.transfermarkt.com/player-name/profil/spieler/{metadata['player_transfermarkt_id']}")

Transfermarkt.com Link: https://www.transfermarkt.com/player-name/profil/spieler/951357


In [46]:
from langchain_core.documents.base import Document

def print_texts(doc: Document):
    print(doc.page_content)

In [48]:
# get original texts
print_texts(retrived_documents[0])


 Wieder als rechter Verteidiger eingesetzt, zeigte er eine beständige Leistung über die gesamte Spielzeit.
Defensiv sehr solide, ließ kaum Flanken und gegnerische Durchbrüche zu.
Offensiv setzte er Akzente durch präzise Flanken und gute Vorstöße.
Seine Ausdauer und Einsatzbereitschaft waren erneut beeindruckend.
Seine Endgeschwindigkeit bleibt eine Schwäche, die gegen schnelle Gegenspieler problematisch sein könnte.
Mit 25 Jahren hat er noch Verbesserungspotenzial.
 


In [55]:
irrelevant_query = "Ich will einen Kuchen backen. Welche Rezepte kannst du mir vorschlagen?"
res = rag_chain.invoke(irrelevant_query)
res

" I'm sorry for any confusion, but the context provided doesn't contain information about recipes for baking a cake. Here are some popular cake recipes you might find interesting:\n\n1. Chocolate Cake: This classic recipe usually requires 2 cups of sugar, 1-3/4 cups all-purpose flour, 3/4 cup unsweetened cocoa powder, 1-1/2 teaspoons baking soda, 1-1/2 teaspoons baking powder, 1 teaspoon salt, 2 eggs, 1 cup milk, 1/2 cup vegetable oil, 2 teaspoons vanilla extract, and 1 cup boiling water.\n\n2. Vanilla Cake: A versatile recipe that requires 1-3/4 cups granulated sugar, 2-1/2 cups all-purpose flour, 3-1/2 teaspoons baking powder, 1 teaspoon salt, 1 cup milk, 2 eggs, 2 teaspoons vanilla extract, and 1/2 cup unsalted butter (melted).\n\n3. Carrot Cake: A sweet and flavorful cake that usually requires 2 cups all-purpose flour, 2 cups sugar, 2 teaspoons baking soda, 1-1/2 teaspoons salt, 1-1/2 teaspoons cinnamon, 4 eggs, 3 cups grated carrots, 1 cup vegetable oil, 3 tablespoons orange juice

In [60]:
# Demonstrate retrival based on metadata
other_retriver = vectorstore.as_retriever(search_kwargs={"expr": 'scout_id == "3456"'})
expr_res = other_retriver.invoke(query)
expr_res

[Document(page_content='\n Kam als Torwart zum Einsatz und zeigte eine solide Leistung.\nParierte mehrere Schüsse gekonnt und bewahrte seine Mannschaft vor einem Rückstand.\nBei hohen Bällen wirkte er jedoch unsicher und machte Fehler in der Strafraumbeherrschung.\nSein Abschlag war stark, aber das Passspiel im Aufbau eher schwach.\nMit 28 Jahren hat er sich gut entwickelt, aber es bleibt Raum für Verbesserungen.\n ', metadata={'id': 450236041423488374, 'scout_id': '3456', 'player_id': '8e1234567890abcdef123456', 'player_transfermarkt_id': '159486', 'grade_rating': 0.6000000238418579, 'grade_potential': 0.699999988079071}),
 Document(page_content='\n Als Torwart zeigte er erneut eine solide Leistung, jedoch mit Schwächen bei hohen Bällen.\nParierte mehrere schwierige Schüsse und bewahrte seine Mannschaft vor einem frühen Rückstand.\nSeine Unsicherheiten bei der Strafraumbeherrschung blieben bestehen, was zu unnötigen Risiken führte.\nSein Passspiel war diesmal besser, insbesondere bei 